In [4]:
!pip install -q transformers torchaudio ffmpeg-python

In [5]:
from google.colab import files
uploaded = files.upload()

Saving video1705_01.mp4 to video1705_01 (1).mp4


In [6]:
import ffmpeg
import os

video_file = next(iter(uploaded))  # lấy tên file vừa upload

# Tách file âm thanh từ video, lưu dưới dạng .wav
audio_file = "extracted_audio.wav"
ffmpeg.input(video_file).output(audio_file, ac=1, ar='16000').run(overwrite_output=True)

(None, None)

In [7]:
import torchaudio

waveform, sr = torchaudio.load(audio_file)
print(f"Waveform shape: {waveform.shape}, Sample Rate: {sr}")

Waveform shape: torch.Size([1, 8134043]), Sample Rate: 16000


In [8]:
from transformers import AutoProcessor, WhisperForConditionalGeneration
import torch

def run_whisper(audio_path, model_name="openai/whisper-medium"):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    processor = AutoProcessor.from_pretrained(model_name)
    model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)

    waveform, sr = torchaudio.load(audio_path)
    if sr != 16000:
        waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    inputs = processor(waveform.squeeze(), sampling_rate=16000, return_tensors="pt").to(device)

    with torch.no_grad():
        predicted_ids = model.generate(inputs["input_features"])
    transcript = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    return transcript

In [9]:
transcript = run_whisper(audio_file)
print("📄 Transcription Result:\n")
print(transcript)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


📄 Transcription Result:

 Ngày 16 tháng 5, Phòng Giáo dục và Đào tạo hiện Vũng Liêm, tỉnh Vĩnh Lâm, Thông tin đã có báo cáo về việc xử lý sai phạm đối với bạp PTBH, hiệu trưởng một trường mẫu giáo trên địa bản. Trước đó, vào cuối tháng 12 năm 2024, Phòng Giáo dục và Đào tạo hiện Vũng Liêm nhận được hai hình ảnh chụp 3H tay cầm một biên bản bằng giấy cắt tông với nội dung.
